In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
ids = np.array(test_data['Id'])
strs = data['Sequence']
test_strs = test_data['Sequence']
sequences = []
test_sequences = []
for line in strs:
    sequences.append([int(i) for i in line.split(',')])
for line in test_strs:
    test_sequences.append([int(i) for i in line.split(',')])

total = len(sequences)
print('Total sequences count: {0}'.format(total))

Total sequences count: 113845


In [4]:
def has_property_percent(f):
    n = 0
    for sequence in sequences:
        if (f(sequence)):
            n += 1
    return (n * 100. / total, n)

def print_percentage(f):
    percent, num = has_property_percent(f)
    print('{0}% ({1})'.format(percent, num))
    
def print_sequences(f):
    for sequence in sequences:
        if (f(sequence)):
            print(sequence, '\n')

In [5]:
def create_dict_percent(condition):
    my_dict = {}
    for i in range(0, 100):
        my_dict[i] = 0
    
    for sequence in sequences:
        sequence = np.array(sequence)
        mask = condition(sequence)
        percent_of_appr = len(sequence[mask]) * 100. / len(sequence)
        key = int(percent_of_appr)
        my_dict[key] += 1
    return my_dict

def plot_dict(d):
    x = np.arange(100)
    plt.bar(x, list(d.values()), width = [1] * len(x))
    labels = []
    for i in range(0, len(x)):
        if i % 10 == 0:
            labels.append(str(i) + '%')
        else:
            labels.append('')
    plt.xticks(x, labels)
    plt.ylabel('number of sequences')
    plt.show()   
    
def explore_by_dict(condition):
    plot_dict(create_dict_percent(condition))

In [6]:
# 1. static
def is_static(sequence):
    return len(np.unique(sequence)) == 1

print_percentage(is_static)

0.01932452018094778% (22)


In [7]:
# 12. арифметическая прогрессия
def is_arithmetic_progression(sequence):
    return len(np.unique(np.diff(sequence))) == 1

print_percentage(is_arithmetic_progression)

0.31534103386182966% (359)


In [8]:
# 13. геометрическая прогрессия
def is_geometric_progression(sequence):
    if (0 in sequence or len(sequence) < 2):
        return False
    q = sequence[1] * 1. / sequence[0]
    for i in range(2, len(sequence)):
        if sequence[i] * 1. / sequence[i - 1] != q:
            return False
    return True

print_percentage(is_geometric_progression)

0.1405419649523475% (160)


In [9]:
# 26. Содержит только однозначные числа
def contains_only_digits(sequence):
    unique_elements = np.unique(sequence)
    if len(unique_elements) > 10:
        return False
    for x in unique_elements:
        if x < 0 or x > 9:
            return False
    return True

print_percentage(contains_only_digits)

7.562914488998199% (8610)


In [10]:
def handle_arithmetic_progression(sequence):
    if len(sequence) <= 1:
        return None
    if is_arithmetic_progression(sequence):
        last = len(sequence) - 1
        d = sequence[last] - sequence[last - 1]
        return sequence[last] + d    
    else:
        return None

def handle_geometric_progression(sequence):
    if len(sequence) <= 1:
        return None
    if is_geometric_progression(sequence):
        last = len(sequence) - 1
        r = round(sequence[last] * sequence[last] // sequence[last - 1])
        return r
    else:
        return None

from numpy import linalg as la

def handle_linear_recurrence(sequence, order):
    if len(sequence) <= 2 * order:
        return None
    a = []
    for i in range(0, order):
        a.append(sequence[i:order + i])
    b = sequence[order:2*order]
    try:
        c = la.solve(a, b)
    except:
        return None
    for i in range(order, len(sequence) - order):
        if np.dot(c, sequence[i:order + i]) != sequence[order + i]:
            return None
    return np.dot(c, sequence[-order:])

def newton_interpolation(sequence, order):
    if (len(sequence) <= order + 1):
        return None
    h = 1
    
    #count coeffs
    div_diff_old = sequence[:order + 1].copy()
    div_diff_new = []
    c = [sequence[0]]
    for i in range(0, order):
        div_diff_new = []
        for j in range(0, order - i):
            div_diff_new.append(div_diff_old[j + 1] - div_diff_old[j])
        div_diff_old = div_diff_new.copy()
        c.append(div_diff_new[0] / np.math.factorial(i + 1))
    if (c[len(c) - 1] == 0):
        return None
        
    #check other points and predict next if everything is ok
    for i in range(order, len(sequence) + 1):
        next_point = 0
        q = (i + 1 - 1) / h
        for j in range(0, len(c)):
            mn = 1
            for k in range(0, j):
                mn *= q - k
            next_point += c[j] * mn
        if i == len(sequence):
            return next_point
        if next_point != sequence[i]:
            return None
        
def divide_accurately(numerator, denominator, accuracy):
    numerator *= 10 ** (accuracy + 1)
    fract = numerator // denominator
    digits_after_point = []
    while fract > 0:
        digits_after_point.append(fract % 10)
        fract = fract // 10
    for i in range(len(digits_after_point), accuracy + 1):
        digits_after_point.append(0)
    digits_after_point = np.fliplr([np.array(digits_after_point)])[0]
    return digits_after_point
        

def get_possible_int(sequence):
    denominator = 0
    mn = 1
    eps = 0.000001
    for x in sequence:
        mn *= 0.1
        denominator += mn * x
    if np.abs((1 / denominator) - round(1 / denominator)) < eps:
        return round(1 / denominator)
    return None

def handle_decimal_expansion(sequence):
    if (len(sequence) <= 2):
        return None
    if (contains_only_digits(sequence) == False):
        return None
    possible_int = get_possible_int(sequence)
    if possible_int is None:
        return None
    possible_float = divide_accurately(1, possible_int, len(sequence) + 2)
    for i, x in enumerate(sequence):
        if x != possible_float[i]:
            return None
    return possible_float[len(sequence)]

def handle_partial_sums(sequence):
    if len(sequence) == 0:
        return None
    initial_sequence = np.hstack((np.array(sequence[0]), np.diff(sequence)))
    next_element = apply_determinate_algorithms(initial_sequence)
    if next_element is None:
        return None
    return sequence[-1] + next_element

def apply_determinate_algorithms(sequence):
    pr = handle_arithmetic_progression(sequence)
    if pr is not None:
        return pr
    else:
        pr = handle_geometric_progression(sequence)
        if pr is not None:
            return pr
        else:
            found_model = False
            for q in range(2, len(sequence) // 2):
                pr = handle_linear_recurrence(sequence, q)
                if pr is not None:
                    return pr
            for order in range(1, 10):
                pr = newton_interpolation(sequence, order)
                if pr is not None:
                    return pr
            pr = handle_decimal_expansion(sequence)
            if pr is not None:
                return pr
    return None

def handle_markov(sequence, depth, most_possibles):
    length = len(sequence)
    if length >= depth:
        prefix = tuple(sequence[(length - depth):])
        if prefix in most_possibles:
            return most_possibles[prefix]
    return None

def handle_linear_regression(sequence):
    if len(sequence) < 5:
        return (None, 0)
    n_train = len(sequence) // 2
    if len(sequence) >= 25:
        n_train = 17
    x_train = []
    y_train = []
    y_test = []
    for i in range(len(sequence) - n_train):
        y_test.append(sequence[i+n_train])
    y_test = np.array(y_test)
    scaler = MinMaxScaler(feature_range=(0, 1))
    sequence = scaler.fit_transform(np.array(sequence).reshape(-1, 1)).reshape(-1)
    for i in range(len(sequence) - n_train):
        x_train.append(sequence[i:i+n_train])
        y_train.append(sequence[i+n_train])

    x_train = np.array(x_train)
    y_train = np.array(y_train)

    regressor = LinearRegression()
    regressor.fit(x_train, y_train)
    pred = round(scaler.inverse_transform(regressor.predict(np.array(sequence[-n_train:]).reshape(1, -1))[0])[0][0])
    validation = [round(item) for item in scaler.inverse_transform(regressor.predict(x_train).reshape(1, -1))[0]]

    sure = sum(y_test == validation)
    sure /= len(x_train)
    return (pred, sure)

In [11]:
n_train = 10

short_rf_regressors = {}
short_x_trains = {}
short_y_trains = {}
qualities = {}
scalers = {}

train_sequences = sequences

for i, sequence in enumerate(np.hstack((train_sequences, test_sequences))):
    if i % 10000 == 0:
        print('adding sequence', i)
    numbers = sequence
    if len(numbers) == 1:
        continue
    if len(numbers) < n_train + 1:
        short_n_train = len(numbers) // 2
    else:
        short_n_train = n_train
    if short_n_train not in short_x_trains:
        short_x_trains[short_n_train] = []
        short_y_trains[short_n_train] = []
    for i in range(len(numbers) - short_n_train - 1):
        short_x_trains[short_n_train].append(numbers[i:i + short_n_train])
        short_y_trains[short_n_train].append(numbers[i + short_n_train])

print('all keys: ', short_x_trains.keys())

adding sequence 0
adding sequence 10000
adding sequence 20000
adding sequence 30000
adding sequence 40000
adding sequence 50000
adding sequence 60000
adding sequence 70000
adding sequence 80000
adding sequence 90000
adding sequence 100000
adding sequence 110000
adding sequence 120000
adding sequence 130000
adding sequence 140000
adding sequence 150000
adding sequence 160000
adding sequence 170000
adding sequence 180000
adding sequence 190000
adding sequence 200000
adding sequence 210000
adding sequence 220000
all keys:  dict_keys([10, 3, 4, 2, 1, 5])


In [19]:
class RandomForestPredictor:
    
    def __init__(self, n_estimators, x_train, y_train, x_test):
        print('start init')
        x_train_unique = np.unique(x_train)
        print('found x train unique')
        x_test_unique = np.unique(x_test)
        print('found x test unique')
        x_arr = np.concatenate((x_train_unique, x_test_unique))
        print('concatenated')
        x_uniques = np.unique(x_arr)
        print('found x unique', len(x_uniques))
        y_uniques = np.unique(y_train)
        print('found y unique', len(y_uniques))
        x_to_indices = {}
        x_from_indices = {}
        y_to_indices = {}
        y_from_indices = {}
        
        n_train = len(x_train[0])
        for i, x in enumerate(x_uniques):
            x_to_indices[x] = i
            x_from_indices[i] = x
        np.save('x_to_indices{0}'.format(n_train), x_to_indices)
        np.save('x_from_indices{0}'.format(n_train), x_from_indices)
        for i, y in enumerate(y_uniques):
            y_to_indices[y] = i
            y_from_indices[i] = y
        np.save('y_to_indices{0}'.format(n_train), y_to_indices)
        np.save('y_from_indices{0}'.format(n_train), y_from_indices)
        self.classifier = RandomForestClassifier(n_estimators)
        print('finish init')

    def fit(self, x_train, y_train):
        print('start fit')
        self.classifier.fit(np.array(self.convert_x_to_indices(x_train)), np.array([self.convert_y_to_indices(y_train, len(x_train[0])) for y in y_train]))
        print('finish fit')

    def convert_x_to_indices(self, x_train):
        result = []
        x_to_indices = np.load('x_to_indices{0}.npy'.format(len(x_train[0]))).item()
        for seq in x_train:
            result.append([x_to_indices[x] for x in seq])
        return np.array(result)
        
    def convert_y_to_indices(self, y_train, n_train):
        result = []
        y_to_indices = np.load('y_to_indices{0}.npy'.format(n_train)).item()
        return np.array([y_to_indices[y] for y in y_train])

    def convert_y_from_indices(self, y_train, n_train):
        result = []
        y_from_indices = np.load('y_from_indices{0}.npy'.format(n_train)).item()
        return np.array([y_from_indices[y] for y in y_train])

    def predict(self, x_test):
        print('start predict')
        predicted = []
        for estimator in self.classifier.estimators_:
            cur_pred = np.array([self.convert_y_from_indices(y, len(x_test[0])) for y in estimator.predict(self.convert_x_to_indices(x_test))]).reshape(-1, 1)
            if len(predicted) == 0:
                predicted = cur_pred
            else:
                predicted = np.append(predicted, cur_pred, axis=1)
        result = []
        for options in predicted:
            result.append(round(np.sum(options) / len(options)))
        print('finish predict')
        return result

In [13]:
test_by_lens = {}
for sequence in test_sequences:
    if len(sequence) == 1:
        cur_len = 1
    elif len(sequence) < n_train:
        cur_len = len(sequence) // 2
    else:
        cur_len = n_train
    if cur_len not in test_by_lens:
        test_by_lens[cur_len] = []
    for i in range(len(sequence) - cur_len - 1):
        test_by_lens[cur_len].append(sequence[i:i + cur_len])

In [14]:
x_to_indices = {}
for i in range(10):
    x_to_indices[i] = i + 10
print(x_to_indices)
np.save('x_to_indices', x_to_indices)
x_to_indices = np.load('x_to_indices.npy').item()
print(x_to_indices)

{0: 10, 1: 11, 2: 12, 3: 13, 4: 14, 5: 15, 6: 16, 7: 17, 8: 18, 9: 19}
{0: 10, 1: 11, 2: 12, 3: 13, 4: 14, 5: 15, 6: 16, 7: 17, 8: 18, 9: 19}


In [ ]:
for short_n_train in short_x_trains:
    print('fitting and checking quality for ', short_n_train)
    cur_x_train = np.array(short_x_trains[short_n_train])
    cur_len = len(cur_x_train[0])
    cur_y_train = np.array(short_y_trains[short_n_train])
    cur_y_test = np.copy(cur_y_train)
    #print(cur_x_train.shape)
    #print(test_by_lens[short_n_train])
    cur_regressor = RandomForestPredictor(25, cur_x_train, cur_y_train, test_by_lens[short_n_train])
    
    cur_regressor.fit(cur_x_train, cur_y_train)
    short_rf_regressors[short_n_train] = cur_regressor
    validation = cur_regressor.predict(cur_x_train)
#     print(validation[:10])
#     print(cur_y_test[:10])
    qualities[short_n_train] = sum(validation == cur_y_test) / len(cur_x_train)
    print('{0}: {1}'.format(short_n_train, qualities[short_n_train]))

fitting and checking quality for  10
start init
found x train unique
found x test unique
concatenated
found x unique 847095
found y unique 779629
finish init
start fit


In [ ]:
def handle_random_forest(sequence, threshold):
    cur_n_train = 0
    if len(sequence) == 1:
        cur_n_train = 1
    elif len(sequence) < n_train:
        cur_n_train = len(sequence) // 2
    else:
        cur_n_train = 10
    print(qualities[cur_n_train])
    if qualities[cur_n_train] >= 0.7:
        cur_x_test = scalers[cur_n_train].transform(np.array(sequence).reshape(-1, 1)).reshape(-1)
        return [round(item) for item in scaler.inverse_transform(short_rf_regressors[cur_n_train].predict(cur_x_test).reshape(1, -1))[0]]
    else:
        return None

In [ ]:
predicted = [None] * len(ids)
for i, sequence in enumerate(test_sequences):
    if i % 1000 == 0:
        print('sequence', i)
    predicted[i] = apply_determinate_algorithms(sequence)
    if predicted[i] is None:
        predicted[i] = handle_partial_sums(sequence)
        if predicted[i] is None:
            (pred, sure) = handle_linear_regression(sequence)
            if sure >= 0.7:
                predicted[i] = pred
            if predicted[i] is None:
                predicted[i] = handle_random_forest(sequence)
                
depth = 30
for cur_depth in range(depth, -1, -1):
    print('cur_depth', cur_depth)
    most_possibles = np.load('most_possibles{0}.npy'.format(cur_depth)).item()
    for i, sequence in enumerate(test_sequences):
        if predicted[i] is None:
            predicted[i] = handle_markov(sequence, cur_depth, most_possibles)
for i, x in enumerate(predicted):
    if x is None:
        predicted[i] = 0

In [ ]:
ids = np.array(test_data['Id'])
with open('submission50.csv', 'w') as f:
    f.write('Id,Last\n')
    for i, x in enumerate(predicted):
        f.write('%d,%d\n' % (ids[i], x))